<a href="https://colab.research.google.com/github/olivia-sp/smart-sv-dev/blob/main/ssv_getRoadPOI_v0_0_olivia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [192]:
#도로정보DB - https://data.gg.go.kr/portal/data/dataset/searchDatasetPage.do
#GPS 데이터 기반 연산 로직 개발
#ver0.1 - 현재 위치 반경 계산 및 성남시 분당구 도로정보 필터링_신호등위치, 횡단보도위치

In [193]:
import pandas as pd
import os
import numpy as np

#DB(csv)파일읽어오기

##신호등위치(성남시분당구)

In [176]:
import os

if os.path.isfile('/content/drive/MyDrive/Colab Notebooks/data_set/신호등현황(개방표준).csv') == True:
  print('already')
else:
  #os.mkdir('data_set')
  %cd data_set
  !wget https://raw.githubusercontent.com/olivia-sp/smart-sv-dev/main/data_set/신호등현황(개방표준).csv

% cd /content


already
/content


In [177]:
CURR_PATH = os.getcwd()

os.listdir(CURR_PATH)
#DATA_PATH = os.path.join(CURR_PATH,'data_set')
RAW_PATH = CURR_PATH + '/drive/MyDrive/Colab Notebooks/data_set/신호등현황(개방표준).csv'



In [178]:
raw_data_traffLgt = pd.read_csv(RAW_PATH, index_col=0, encoding='cp949')
raw_data_traffLgt.info()
raw_data_traffLgt.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2972 entries, 9 to 9
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   시군구명           2972 non-null   object 
 1   도로종류           2972 non-null   object 
 2   도로노선번호         1430 non-null   object 
 3   도로노선명          2972 non-null   object 
 4   도로노선방향         2972 non-null   int64  
 5   소재지도로명주소       564 non-null    object 
 6   소재지지번주소        1420 non-null   object 
 7   위도             1420 non-null   float64
 8   경도             1420 non-null   float64
 9   신호기설치방식        2972 non-null   int64  
 10  도로형태           2972 non-null   int64  
 11  주도로여부          2972 non-null   object 
 12  신호등관리번호        2972 non-null   object 
 13  신호등구분          2972 non-null   int64  
 14  신호등색종류         2972 non-null   int64  
 15  신호등화방식         2972 non-null   int64  
 16  신호등화순서         2972 non-null   object 
 17  신호등화시간         2972 non-null   object 
 18  광원종류       

시군구명                0
도로종류                0
도로노선번호           1542
도로노선명               0
도로노선방향              0
소재지도로명주소         2408
소재지지번주소          1552
위도               1552
경도               1552
신호기설치방식             0
도로형태                0
주도로여부               0
신호등관리번호             0
신호등구분               0
신호등색종류              0
신호등화방식              0
신호등화순서              0
신호등화시간              0
광원종류                0
신호제어방식              0
신호시간결정방식            0
점멸등운영여부             0
점멸등운영시작시각           0
점멸등운영종료시각           0
보행자작동신호기유무          0
잔여시간표시기유무           0
시각장애인용음향신호기유무       0
도로안내표지일련번호       2972
관리기관명               0
관리기관전화번호            0
데이터기준일자             0
dtype: int64

In [179]:
if 'index' not in raw_data_traffLgt.columns:
  raw_data_traffLgt.reset_index(inplace=True)

In [180]:
#공백제거
raw_data_traffLgt["시군구명"] = raw_data_traffLgt["시군구명"].str.replace(" ", "")
raw_data_traffLgt.head(3)

,시도명,시군구명,도로종류,도로노선번호,도로노선명,도로노선방향,소재지도로명주소,소재지지번주소,위도,경도,...,점멸등운영여부,점멸등운영시작시각,점멸등운영종료시각,보행자작동신호기유무,잔여시간표시기유무,시각장애인용음향신호기유무,도로안내표지일련번호,관리기관명,관리기관전화번호,데이터기준일자
0,9,광명시,시도,NaN,목감로,3,NaN,NaN,NaN,NaN,...,Y,00:00,00:00,N,N,N,NaN,광명시 도시교통과,02-2680-2585,2022-04-29
1,9,광명시,시도,NaN,목감로,3,NaN,NaN,NaN,NaN,...,Y,00:00,00:00,N,N,N,NaN,광명시 도시교통과,02-2680-2585,2022-04-29
2,9,광명시,시도,NaN,목감로,3,NaN,NaN,NaN,NaN,...,Y,00:00,00:00,N,N,N,NaN,광명시 도시교통과,02-2680-2585,2022-04-29


In [181]:
filter_city=raw_data_traffLgt['시군구명'] == '성남시분당구'
df_traffLgt_SNcity = raw_data_traffLgt[filter_city]

In [182]:
df_traffLgt_SNcity.reset_index(inplace=True, drop=True)

In [183]:
df_traffLgt_SNcity.head(2)

,시도명,시군구명,도로종류,도로노선번호,도로노선명,도로노선방향,소재지도로명주소,소재지지번주소,위도,경도,...,점멸등운영여부,점멸등운영시작시각,점멸등운영종료시각,보행자작동신호기유무,잔여시간표시기유무,시각장애인용음향신호기유무,도로안내표지일련번호,관리기관명,관리기관전화번호,데이터기준일자
0,9,성남시분당구,시도,판교원로 82번길,판교원로 82번길,3,경기도 성남시 분당구 판교원로82번길 60,경기도 성남시 분당구 운중동 916,37.384058,127.072437,...,Y,00:00,05:30,N,N,N,NaN,성남시청,031-729-3673,2018-08-31
1,9,성남시분당구,시도,판교역로,판교역로,3,경기도 성남시 분당구 판교역로 188,경기도 성남시 분당구 삼평동 642,37.397392,127.109988,...,N,00:00,00:00,N,Y,N,NaN,성남시청,031-729-3673,2018-08-31


##횡단보도위치(성남시분당구)

In [204]:
import os

if os.path.isfile('/content/drive/MyDrive/Colab Notebooks/data_set/횡단보도현황(개방표준).csv') == True:
  print('already')
else:
  #os.mkdir('data_set')
  %cd data_set
  !wget https://raw.githubusercontent.com/olivia-sp/smart-sv-dev/main/data_set/횡단보도현황(개방표준).csv

% cd /content


already
/content


In [205]:
CURR_PATH = os.getcwd()

os.listdir(CURR_PATH)
#DATA_PATH = os.path.join(CURR_PATH,'data_set')
RAW_PATH = CURR_PATH + '/drive/MyDrive/Colab Notebooks/data_set/횡단보도현황(개방표준).csv'



In [206]:
raw_data_crossWlk = pd.read_csv(RAW_PATH, index_col=0, encoding='cp949')
raw_data_crossWlk.info()
raw_data_crossWlk.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5003 entries, 9 to 9
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   시군구명        5003 non-null   object 
 1   도로명         5003 non-null   object 
 2   소재지도로명주소    575 non-null    object 
 3   소재지지번주소     2710 non-null   object 
 4   횡단보도관리번호    5003 non-null   object 
 5   횡단보도종류      5003 non-null   int64  
 6   자전거횡단도겸용여부  670 non-null    object 
 7   고원식적용여부     5003 non-null   object 
 8   위도          2710 non-null   float64
 9   경도          2710 non-null   float64
 10  차로수         5003 non-null   int64  
 11  횡단보도폭       5003 non-null   float64
 12  횡단보도연장      5003 non-null   float64
 13  보행자신호등유무    5003 non-null   object 
 14  보행자작동신호기유무  907 non-null    object 
 15  음향신호기설치여부   907 non-null    object 
 16  녹색신호시간      104 non-null    float64
 17  적색신호시간      0 non-null      float64
 18  교통섬유무       5003 non-null   object 
 19  보도턱낮춤여부     1818 non-null   ob

시군구명             0
도로명              0
소재지도로명주소      4428
소재지지번주소       2293
횡단보도관리번호         0
횡단보도종류           0
자전거횡단도겸용여부    4333
고원식적용여부          0
위도            2293
경도            2293
차로수              0
횡단보도폭            0
횡단보도연장           0
보행자신호등유무         0
보행자작동신호기유무    4096
음향신호기설치여부     4096
녹색신호시간        4899
적색신호시간        5003
교통섬유무            0
보도턱낮춤여부       3185
점자블록유무           0
집중조명시설유무      3955
관리기관명            0
관리기관전화번호         0
데이터기준일자          0
dtype: int64

In [207]:
if 'index' not in raw_data_crossWlk.columns:
  raw_data_crossWlk.reset_index(inplace=True)

In [208]:
#공백제거
raw_data_crossWlk["시군구명"] = raw_data_crossWlk["시군구명"].str.replace(" ", "")
raw_data_crossWlk.head(3)

,시도명,시군구명,도로명,소재지도로명주소,소재지지번주소,횡단보도관리번호,횡단보도종류,자전거횡단도겸용여부,고원식적용여부,위도,...,음향신호기설치여부,녹색신호시간,적색신호시간,교통섬유무,보도턱낮춤여부,점자블록유무,집중조명시설유무,관리기관명,관리기관전화번호,데이터기준일자
0,9,광명시,오리로,NaN,NaN,2020-180,1,NaN,N,NaN,...,N,NaN,NaN,N,NaN,Y,NaN,광명시 도시교통과,02-2680-2586,2022-04-15
1,9,광명시,오리로,NaN,NaN,2020-181,1,NaN,N,NaN,...,Y,NaN,NaN,N,NaN,Y,NaN,광명시 도시교통과,02-2680-2586,2022-04-15
2,9,광명시,광명로865번길,NaN,NaN,2020-246,1,NaN,N,NaN,...,N,NaN,NaN,N,NaN,Y,NaN,광명시 도시교통과,02-2680-2586,2022-04-15


In [209]:
filter_city=raw_data_crossWlk['시군구명'] == '성남시분당구'
df_crossWlk_SNcity = raw_data_crossWlk[filter_city]

In [210]:
df_crossWlk_SNcity.reset_index(inplace=True, drop=True)

In [211]:
df_crossWlk_SNcity.head(2)

,시도명,시군구명,도로명,소재지도로명주소,소재지지번주소,횡단보도관리번호,횡단보도종류,자전거횡단도겸용여부,고원식적용여부,위도,...,음향신호기설치여부,녹색신호시간,적색신호시간,교통섬유무,보도턱낮춤여부,점자블록유무,집중조명시설유무,관리기관명,관리기관전화번호,데이터기준일자
0,9,성남시분당구,운중로 243번길,경기도 성남시 분당구 운중로243번길 12,경기도 성남시 분당구 판교동 612-4번지,-,4,NaN,N,37.390521,...,NaN,NaN,NaN,N,Y,Y,Y,성남시청,031-729-3672,2020-06-30
1,9,성남시분당구,동판교로,"경기도 성남시 분당구 동판교로 212 (삼평동, 봇들마을6단지아파트)",경기도 성남시 분당구 삼평동 728번지 봇들마을6단지아파트,-,4,NaN,N,37.401425,...,NaN,NaN,NaN,N,Y,Y,Y,성남시청,031-729-3672,2020-06-30


##횡단보도위치(성남시분당구)

In [227]:
import os

if os.path.isfile('/content/drive/MyDrive/Colab Notebooks/data_set/횡단보도현황(개방표준).csv') == True:
  print('already')
else:
  #os.mkdir('data_set')
  %cd data_set
  !wget https://raw.githubusercontent.com/olivia-sp/smart-sv-dev/main/data_set/횡단보도현황(개방표준).csv

% cd /content


already
/content


In [228]:
CURR_PATH = os.getcwd()

os.listdir(CURR_PATH)
#DATA_PATH = os.path.join(CURR_PATH,'data_set')
RAW_PATH = CURR_PATH + '/drive/MyDrive/Colab Notebooks/data_set/횡단보도현황(개방표준).csv'



In [229]:
raw_data_crossWlk = pd.read_csv(RAW_PATH, index_col=0, encoding='cp949')
raw_data_crossWlk.info()
raw_data_crossWlk.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5003 entries, 9 to 9
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   시군구명        5003 non-null   object 
 1   도로명         5003 non-null   object 
 2   소재지도로명주소    575 non-null    object 
 3   소재지지번주소     2710 non-null   object 
 4   횡단보도관리번호    5003 non-null   object 
 5   횡단보도종류      5003 non-null   int64  
 6   자전거횡단도겸용여부  670 non-null    object 
 7   고원식적용여부     5003 non-null   object 
 8   위도          2710 non-null   float64
 9   경도          2710 non-null   float64
 10  차로수         5003 non-null   int64  
 11  횡단보도폭       5003 non-null   float64
 12  횡단보도연장      5003 non-null   float64
 13  보행자신호등유무    5003 non-null   object 
 14  보행자작동신호기유무  907 non-null    object 
 15  음향신호기설치여부   907 non-null    object 
 16  녹색신호시간      104 non-null    float64
 17  적색신호시간      0 non-null      float64
 18  교통섬유무       5003 non-null   object 
 19  보도턱낮춤여부     1818 non-null   ob

시군구명             0
도로명              0
소재지도로명주소      4428
소재지지번주소       2293
횡단보도관리번호         0
횡단보도종류           0
자전거횡단도겸용여부    4333
고원식적용여부          0
위도            2293
경도            2293
차로수              0
횡단보도폭            0
횡단보도연장           0
보행자신호등유무         0
보행자작동신호기유무    4096
음향신호기설치여부     4096
녹색신호시간        4899
적색신호시간        5003
교통섬유무            0
보도턱낮춤여부       3185
점자블록유무           0
집중조명시설유무      3955
관리기관명            0
관리기관전화번호         0
데이터기준일자          0
dtype: int64

In [230]:
if 'index' not in raw_data_crossWlk.columns:
  raw_data_crossWlk.reset_index(inplace=True)

In [231]:
#공백제거
raw_data_crossWlk["시군구명"] = raw_data_crossWlk["시군구명"].str.replace(" ", "")
raw_data_crossWlk.head(3)

,시도명,시군구명,도로명,소재지도로명주소,소재지지번주소,횡단보도관리번호,횡단보도종류,자전거횡단도겸용여부,고원식적용여부,위도,...,음향신호기설치여부,녹색신호시간,적색신호시간,교통섬유무,보도턱낮춤여부,점자블록유무,집중조명시설유무,관리기관명,관리기관전화번호,데이터기준일자
0,9,광명시,오리로,NaN,NaN,2020-180,1,NaN,N,NaN,...,N,NaN,NaN,N,NaN,Y,NaN,광명시 도시교통과,02-2680-2586,2022-04-15
1,9,광명시,오리로,NaN,NaN,2020-181,1,NaN,N,NaN,...,Y,NaN,NaN,N,NaN,Y,NaN,광명시 도시교통과,02-2680-2586,2022-04-15
2,9,광명시,광명로865번길,NaN,NaN,2020-246,1,NaN,N,NaN,...,N,NaN,NaN,N,NaN,Y,NaN,광명시 도시교통과,02-2680-2586,2022-04-15


In [232]:
filter_city=raw_data_crossWlk['시군구명'] == '성남시분당구'
df_crossWlk_SNcity = raw_data_crossWlk[filter_city]

In [233]:
df_crossWlk_SNcity.reset_index(inplace=True, drop=True)

In [234]:
df_crossWlk_SNcity.head(2)

,시도명,시군구명,도로명,소재지도로명주소,소재지지번주소,횡단보도관리번호,횡단보도종류,자전거횡단도겸용여부,고원식적용여부,위도,...,음향신호기설치여부,녹색신호시간,적색신호시간,교통섬유무,보도턱낮춤여부,점자블록유무,집중조명시설유무,관리기관명,관리기관전화번호,데이터기준일자
0,9,성남시분당구,운중로 243번길,경기도 성남시 분당구 운중로243번길 12,경기도 성남시 분당구 판교동 612-4번지,-,4,NaN,N,37.390521,...,NaN,NaN,NaN,N,Y,Y,Y,성남시청,031-729-3672,2020-06-30
1,9,성남시분당구,동판교로,"경기도 성남시 분당구 동판교로 212 (삼평동, 봇들마을6단지아파트)",경기도 성남시 분당구 삼평동 728번지 봇들마을6단지아파트,-,4,NaN,N,37.401425,...,NaN,NaN,NaN,N,Y,Y,Y,성남시청,031-729-3672,2020-06-30


#fake GPS 더미

In [17]:
!pip3 install Faker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [85]:
from faker import Faker
fake = Faker('ko_KR')
# for _ in range(5):
#   latlng = fake.local_latlng('KR')
#   print(latlng)

latlng = fake.local_latlng('KR')
print(latlng)
lat = float (latlng[0])
lon = float (latlng[1])
my_latlng = (lat, lon)
print(my_latlng)

('34.8825', '128.62667', 'Sinhyeon', 'KR', 'Asia/Seoul')
(34.8825, 128.62667)


# 현재 위치 반경 계산

In [9]:
!pip3 install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
### 공식문서 예시
# from haversine import haversine, Unit

# lyon = (45.7597, 4.8422) # (lat, lon)
# paris = (48.8567, 2.3508)

# haversine(lyon, paris)
# >> 392.2172595594006  # in kilometers

# haversine(lyon, paris, unit=Unit.MILES)
# >> 243.71201856934454  # in miles

# # you can also use the string abbreviation for units:
# haversine(lyon, paris, unit='mi')
# >> 243.71201856934454  # in miles

# haversine(lyon, paris, unit=Unit.NAUTICAL_MILES)
# >> 211.78037755311516  # in nautical miles

In [173]:
df_traffLgt_SNcity.info()
df_crossWlk_SNcity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 32 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   시도명            414 non-null    int64  
 1   시군구명           414 non-null    object 
 2   도로종류           414 non-null    object 
 3   도로노선번호         414 non-null    object 
 4   도로노선명          414 non-null    object 
 5   도로노선방향         414 non-null    int64  
 6   소재지도로명주소       298 non-null    object 
 7   소재지지번주소        414 non-null    object 
 8   위도             414 non-null    float64
 9   경도             414 non-null    float64
 10  신호기설치방식        414 non-null    int64  
 11  도로형태           414 non-null    int64  
 12  주도로여부          414 non-null    object 
 13  신호등관리번호        414 non-null    object 
 14  신호등구분          414 non-null    int64  
 15  신호등색종류         414 non-null    int64  
 16  신호등화방식         414 non-null    int64  
 17  신호등화순서         414 non-null    object 
 18  신호등화시간    

In [223]:
#판교만도중앙연구소
my_latlng = (37.404704734328, 127.10535530866)

In [175]:
from haversine import haversine, Unit
distance_compare_list = []

## traffic light
for i in range(df_traffLgt_SNcity.shape[0]):
    db_traffLgt = (df_traffLgt_SNcity["위도"][i], df_traffLgt_SNcity["경도"][i])
    result = haversine(db_traffLgt, my_latlng, unit='m')
    if result <= 500:
      distance_compare_list.append(result)
      print('trafficLight', db_traffLgt)
#print(distance_compare_list)

## cross walk
for i in range(df_crossWlk_SNcity.shape[0]):
    db_crosWlk = (df_crossWlk_SNcity["위도"][i], df_crossWlk_SNcity["경도"][i])
    result = haversine(db_crosWlk, my_latlng, unit='m')
    if result <= 500:
      distance_compare_list.append(result)
      print('crossWalk', db_crosWlk)

trafficLight (37.4037259458, 127.0998775385)
trafficLight (37.4045092638, 127.1005562139)
trafficLight (37.4041629952, 127.1051525762)
trafficLight (37.4044202324, 127.1097384586)
trafficLight (37.4033105416, 127.1010177224)
trafficLight (37.4033105416, 127.1010177224)
trafficLight (37.4019750211, 127.1034329076)
trafficLight (37.4045044843, 127.1061808333)
trafficLight (37.400611472, 127.1069321778)
trafficLight (37.4033935026, 127.1092739361)
trafficLight (37.4027630214, 127.1090245731)
crossWalk (37.405076, 127.100406)
crossWalk (37.403169, 127.102582)
crossWalk (37.402375, 127.109682)
crossWalk (37.403942, 127.109649)
crossWalk (37.402307, 127.102577)
crossWalk (37.404494, 127.101492)
crossWalk (37.400852, 127.102599)
crossWalk (37.403902, 127.106456)
crossWalk (37.403183, 127.105057)
crossWalk (37.403178, 127.109643)


In [335]:
from haversine import haversine, Unit
distance_compare_list = []

df_new_traffLgt = pd.DataFrame(columns = df_traffLgt_SNcity.columns)
df_new_crosWlk = pd.DataFrame(columns = df_crossWlk_SNcity.columns)

## traffic light
for i in range(df_traffLgt_SNcity.shape[0]):
    db_traffLgt = (df_traffLgt_SNcity["위도"][i], df_traffLgt_SNcity["경도"][i])
    result = haversine(db_traffLgt, my_latlng, unit='m')
    if result <= 500:
      distance_compare_list.append(result)
      print('trafficLight', db_traffLgt)
      filter_lat=df_traffLgt_SNcity['위도'] == db_traffLgt[0]
      filter_lon=df_traffLgt_SNcity['경도'] == db_traffLgt[1]
      df = df_traffLgt_SNcity[filter_lat & filter_lon]
      df_new_traffLgt = pd.concat([df_new_traffLgt, df], axis = 0)

# cross walk
for i in range(df_crossWlk_SNcity.shape[0]):
    db_crosWlk = (df_crossWlk_SNcity["위도"][i], df_crossWlk_SNcity["경도"][i])
    result = haversine(db_crosWlk, my_latlng, unit='m')
    if result <= 500:
      distance_compare_list.append(result)
      print('crossWalk', db_crosWlk)
      filter_lat=df_crossWlk_SNcity['위도'] == db_crosWlk[0]
      filter_lon=df_crossWlk_SNcity['경도'] == db_crosWlk[1]
      df = df_crossWlk_SNcity[filter_lat & filter_lon]
      df_new_crosWlk = pd.concat([df_new_crosWlk, df], axis = 0)

trafficLight (37.4037259458, 127.0998775385)
trafficLight (37.4045092638, 127.1005562139)
trafficLight (37.4041629952, 127.1051525762)
trafficLight (37.4044202324, 127.1097384586)
trafficLight (37.4033105416, 127.1010177224)
trafficLight (37.4033105416, 127.1010177224)
trafficLight (37.4019750211, 127.1034329076)
trafficLight (37.4045044843, 127.1061808333)
trafficLight (37.400611472, 127.1069321778)
trafficLight (37.4033935026, 127.1092739361)
trafficLight (37.4027630214, 127.1090245731)
crossWalk (37.405076, 127.100406)
crossWalk (37.403169, 127.102582)
crossWalk (37.402375, 127.109682)
crossWalk (37.403942, 127.109649)
crossWalk (37.402307, 127.102577)
crossWalk (37.404494, 127.101492)
crossWalk (37.400852, 127.102599)
crossWalk (37.403902, 127.106456)
crossWalk (37.403183, 127.105057)
crossWalk (37.403178, 127.109643)


In [336]:
df_new_traffLgt

,시도명,시군구명,도로종류,도로노선번호,도로노선명,도로노선방향,소재지도로명주소,소재지지번주소,위도,경도,...,점멸등운영여부,점멸등운영시작시각,점멸등운영종료시각,보행자작동신호기유무,잔여시간표시기유무,시각장애인용음향신호기유무,도로안내표지일련번호,관리기관명,관리기관전화번호,데이터기준일자
15,9,성남시분당구,시도,판교로227번길,판교로227번길,3,경기도 성남시 분당구 판교로255번길 9-22,경기도 성남시 분당구 삼평동 618,37.403726,127.099878,...,Y,00:00,24:00,N,Y,N,NaN,성남시청,031-729-3673,2018-08-31
16,9,성남시분당구,시도,판교로255번길,판교로255번길,3,경기도 성남시 분당구 판교로255번길 21,경기도 성남시 분당구 삼평동 619,37.404509,127.100556,...,Y,00:00,24:00,N,Y,N,NaN,성남시청,031-729-3673,2018-08-31
117,9,성남시분당구,시도,대왕판교로,대왕판교로,3,경기도 성남시 분당구 대왕판교로 700,경기도 성남시 분당구 삼평동 694-1,37.404163,127.105153,...,N,00:00,00:00,N,Y,N,NaN,성남시청,031-729-3673,2018-08-31
160,9,성남시분당구,시도,판교로319번길,판교로319번길,3,경기도 성남시 분당구 판교로319번길 6,경기도 성남시 분당구 삼평동 701,37.404420,127.109738,...,Y,00:00,24:00,N,Y,N,NaN,성남시청,031-729-3673,2018-08-31
230,9,성남시분당구,시도,판교로255번길,판교로255번길,3,경기도 성남시 분당구 판교로 255,경기도 성남시 분당구 삼평동 622,37.403311,127.101018,...,Y,19:00,07:00,N,Y,N,NaN,성남시청,031-729-3673,2018-08-31
231,9,성남시분당구,시도,판교로255번길,판교로255번길,3,경기도 성남시 분당구 판교로 255,경기도 성남시 분당구 삼평동 622,37.403311,127.101018,...,Y,19:00,07:00,N,Y,N,NaN,성남시청,031-729-3673,2018-08-31
230,9,성남시분당구,시도,판교로255번길,판교로255번길,3,경기도 성남시 분당구 판교로 255,경기도 성남시 분당구 삼평동 622,37.403311,127.101018,...,Y,19:00,07:00,N,Y,N,NaN,성남시청,031-729-3673,2018-08-31
231,9,성남시분당구,시도,판교로255번길,판교로255번길,3,경기도 성남시 분당구 판교로 255,경기도 성남시 분당구 삼평동 622,37.403311,127.101018,...,Y,19:00,07:00,N,Y,N,NaN,성남시청,031-729-3673,2018-08-31
232,9,성남시분당구,시도,판교로256번길,판교로256번길,3,경기도 성남시 분당구 판교로256번길 7,경기도 성남시 분당구 삼평동 627,37.401975,127.103433,...,Y,00:00,24:00,N,Y,N,NaN,성남시청,031-729-3673,2018-08-31
238,9,성남시분당구,시도,판교로319번길,판교로319번길,3,경기도 성남시 분당구 판교로289번길 20,경기도 성남시 분당구 삼평동 698,37.404504,127.106181,...,Y,00:00,24:00,N,Y,N,NaN,성남시청,031-729-3673,2018-08-31


In [332]:
df_new_crosWlk

,시도명,시군구명,도로명,소재지도로명주소,소재지지번주소,횡단보도관리번호,횡단보도종류,자전거횡단도겸용여부,고원식적용여부,위도,...,음향신호기설치여부,녹색신호시간,적색신호시간,교통섬유무,보도턱낮춤여부,점자블록유무,집중조명시설유무,관리기관명,관리기관전화번호,데이터기준일자
76,9,성남시분당구,판교로255번길,경기도 성남시 분당구 판교로255번길 21,경기도 성남시 분당구 삼평동 619번지,-,4,NaN,N,37.405076,...,NaN,NaN,NaN,N,Y,Y,Y,성남시청,031-729-3672,2020-06-30
77,9,성남시분당구,판교로255번길,경기도 성남시 분당구 판교로 255,경기도 성남시 분당구 삼평동 622번지,-,4,NaN,N,37.403169,...,NaN,NaN,NaN,N,Y,Y,Y,성남시청,031-729-3672,2020-06-30
149,9,성남시분당구,판교역로,경기도 성남시 분당구 판교로 310,경기도 성남시 분당구 삼평동 686번지,-,4,NaN,N,37.402375,...,NaN,NaN,NaN,N,Y,Y,Y,성남시청,031-729-3672,2020-06-30
168,9,성남시분당구,판교로319번길,경기도 성남시 분당구 판교로319번길 6,경기도 성남시 분당구 삼평동 701번지,-,4,NaN,N,37.403942,...,NaN,NaN,NaN,N,Y,Y,Y,성남시청,031-729-3672,2020-06-30
228,9,성남시분당구,판교로256번길,경기도 성남시 분당구 판교로256번길 7,경기도 성남시 분당구 삼평동 627번지,-,4,NaN,N,37.402307,...,NaN,NaN,NaN,N,Y,Y,Y,성남시청,031-729-3672,2020-06-30
229,9,성남시분당구,판교로255번길,경기도 성남시 분당구 판교로 255,경기도 성남시 분당구 삼평동 622번지,-,4,NaN,N,37.404494,...,NaN,NaN,NaN,N,Y,Y,N,성남시청,031-729-3672,2020-06-30
230,9,성남시분당구,판교로256번길,경기도 성남시 분당구 대왕판교로645번길 36,경기도 성남시 분당구 삼평동 632번지,-,4,NaN,N,37.400852,...,NaN,NaN,NaN,N,Y,Y,Y,성남시청,031-729-3672,2020-06-30
236,9,성남시분당구,판교로319번길,경기도 성남시 분당구 판교로289번길 20,경기도 성남시 분당구 삼평동 698번지,-,4,NaN,N,37.403902,...,NaN,NaN,NaN,N,Y,Y,Y,성남시청,031-729-3672,2020-06-30
237,9,성남시분당구,대왕판교로,경기도 성남시 분당구 판교로 289,경기도 성남시 분당구 삼평동 694번지,-,4,NaN,N,37.403183,...,NaN,NaN,NaN,Y,Y,Y,Y,성남시청,031-729-3672,2020-06-30
360,9,성남시분당구,판교역로,경기도 성남시 분당구 판교로319번길 13,경기도 성남시 분당구 삼평동 691번지,-,4,NaN,N,37.403178,...,NaN,NaN,NaN,N,Y,Y,Y,성남시청,031-729-3672,2020-06-30
